# Perceptron



In [1]:
import numpy as np
np.seterr('ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

# Perceptron Network

In [2]:
# Here I am building a normal forward propogation for a perceptron layer
# and the various function i would use.

In [3]:
#############################################
## Declaring Training Data        ############
#############################################
X_train = np.array([[0,0],[1,0],[0,1],[1,1]])
Y_train = np.array([[1],[0],[0],[0]])



In [4]:
from MultiLayerPerceptron import MultiLayerPerceptron

In [5]:
def accuracy(model , X , Y):
    pred,_ = model.forward(X)
    pred =pred > 0.5
    acc = np.sum(pred == Y)
    acc = float(acc) / Y.shape[0]
    return acc

In [6]:
# Declare a neuron with shape of weights as [shape_of_input,1]
model = MultiLayerPerceptron([2,1],['sigmoid'])
# print(model.layers[1].W)
# print(model.layers[1].b)

(2, 1) (1,)
<function sigmoid at 0x7f0ee8a90378>


In [7]:
# testing the model outputs
pred , _ = model.forward(X_train)
# print(np.sum((pred > 0.5)== Y_train) / Y_train.shape[0])
# accuracy(model , X_train,Y_train)
# X_train[0]

In [8]:
# Checking for testing purposes(BCE should be used here ideally)
from Loss import mean_abs_error,mean_square_error
loss,d_back = mean_square_error(pred,Y_train)
print("The loss is {}\nAnd the error to be back propogated is:\n {}".format(loss , d_back))

The loss is [0.15377332]
And the error to be back propogated is:
 [[-0.125     ]
 [ 0.16124503]
 [ 0.11302115]
 [ 0.14995973]]


In [9]:
from Loss import binary_cross_entropy
loss,d_back = binary_cross_entropy(pred,Y_train)
print("The loss is {}\nAnd the error to be back propogated is:\n {}".format(loss , d_back))

The loss is [3.24625113]
And the error to be back propogated is:
 [[-2.        ]
 [ 2.81674354]
 [ 1.82509925]
 [ 2.49899372]]


In [10]:
# Now we can train the model by iteratively on each datapoint.

layer_list = [2,3, 1]
activation_list = ['sigmoid','sigmoid']
model = MultiLayerPerceptron(layer_list,activation_list)

def train(model ,X_train , Y_train,epochs,alpha =0.1):
    for i in range(epochs):
        act , cache = model.forward(X_train)
        loss,d_back= binary_cross_entropy(act,Y_train)
        acc = accuracy(model ,X_train,Y_train)
        print("The training loss at {}th epoch : {}  Training Accuracy:{}".format(i , loss , acc))
        model.update_gradient(cache,d_back,alpha)        

train(model , X_train,Y_train , 10,alpha = 0.4)
    

(2, 3) (3,)
<function sigmoid at 0x7f0ee8a90378>
(3, 1) (1,)
<function sigmoid at 0x7f0ee8a90378>
The training loss at 0th epoch : [2.28988323]  Training Accuracy:0.75
The training loss at 1th epoch : [2.09854785]  Training Accuracy:0.75
The training loss at 2th epoch : [1.96345838]  Training Accuracy:0.75
The training loss at 3th epoch : [1.85680762]  Training Accuracy:0.75
The training loss at 4th epoch : [1.76139773]  Training Accuracy:0.75
The training loss at 5th epoch : [1.66815315]  Training Accuracy:0.75
The training loss at 6th epoch : [1.57297909]  Training Accuracy:0.75
The training loss at 7th epoch : [1.47463612]  Training Accuracy:0.75
The training loss at 8th epoch : [1.37352834]  Training Accuracy:0.75
The training loss at 9th epoch : [1.27100815]  Training Accuracy:0.75


In [11]:
# Now we can predict the values for unseen data or trained data also
# We can also calculate the accuracy of the model we have trained
accuracy(model , X_train,Y_train)

1.0

# N Bit XOR

Now Lets try working with just a little better data. A n XOR operator. So lets create the dataset for n bit xor.

We would have atmost 2^n data point in this type of data set.BUt we would limit our dataset to a 1000 data points
whichever is smaller.

Then we can divide into training and test set

In [26]:

n = 10
max_datapoint = 10000
datapoints = min(pow(2,n) , max_datapoint)

X = np.zeros((datapoints , n) , dtype=np.int32)
Y = np.zeros((datapoints , 1), dtype=np.int32)

for i in range(datapoints):
    tmp = i
    y_tmp = 0
    for j in range(n-1 , -1 , -1):
        X[i,j] = tmp&1
        y_tmp = y_tmp^X[i,j]
        tmp = tmp>>1
    Y[i] = y_tmp
        
        

In [27]:
# for sanity check lets print one example
ind = 11
print(X[ind] , Y[ind])

[0 0 0 0 0 0 1 0 1 1] [1]


In [28]:
# Lets divide the set in training and testing
div = 0.9
train_n = int(div * datapoints)
X_train = X[:train_n]
Y_train = Y[:train_n]

X_test = X[train_n:]
Y_test = Y[train_n:]
Y_test.shape

(103, 1)

In [29]:
layer_list = [n,8,2,1]
activation_list = ['sigmoid','relu','sigmoid']

model = MultiLayerPerceptron(layer_list,activation_list)

(10, 8) (8,)
<function sigmoid at 0x7f1fe57bd840>
(8, 2) (2,)
<function relu at 0x7f1fc631a620>
(2, 1) (1,)
<function sigmoid at 0x7f1fe57bd840>


In [31]:
train(model,X_train,Y_train , 2000,alpha = 0.01)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [20]:
print("Training accuracy:{}".format(accuracy(model,X_train, Y_train)))

print("Test accuracy:{}".format(accuracy(model,X_test, Y_test)))


Training accuracy:0.500542888165038
Test accuracy:0.49514563106796117
